In [24]:
from functions import *
import os,json
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances

basepath = "/media/ruben/Elements/react-data/iconic/tsv/"

In [2]:
photo = 'AbuGhraib'
sample_size = 10000
texts = Data.Import(photo,sample_size)

 11%|█         | 519/4810 [00:00<00:00, 5186.65it/s]

 tokenizing....


  0%|          | 0/4810 [00:00<?, ?it/s]

 reading languages + subsetting....


100%|██████████| 4810/4810 [01:26<00:00, 55.68it/s] 

after tokenizing and subsetting: 1093 documents left


In [3]:
model = Embeddings.TrainModel(texts)

 training doc2vec model....


In [7]:
umap_model = Cluster.DrUmap(model)

In [79]:
def ClusNum(vectors):
        #word_cosine = cosine_distances(vectors)
        affprop = AffinityPropagation(max_iter=300)
        af = affprop.fit(vectors)
        cluster_centers_indices = af.cluster_centers_indices_
        labels = af.labels_

        no_clusters = len(cluster_centers_indices)
        return no_clusters

In [80]:
ClusNum(umap_model.embedding_)

/home/ruben/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/cluster/_affinity_propagation.py:152: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning)


50

In [81]:
num_clusters = 50
gmm = GaussianMixture(n_components=num_clusters)
gmm.fit(umap_model.embedding_)
cluster_gmm = gmm.predict_proba(umap_model.embedding_)
topic_vectors = Cluster._create_topic_vectors(model,cluster_gmm,num_clusters)
topic_words,topic_word_scores = Cluster._find_topic_words_scores(model,topic_vectors)
data = [[c," ".join(t[:20]),sum([x[c] for x in cluster_gmm])] for c,t in enumerate(topic_words)]
data = pd.DataFrame(data,columns=['cluster','words','prominence'])

In [82]:
data.sort_values('prominence',ascending=False)

,cluster,words,prominence
18,18,etsy tips cookies enable msn shopping ceneval ...,54.013099
8,8,beckett shit fucking theatre fuck me bookmarke...,42.025142
12,12,aclu judge lawsuit caci court release tapes de...,41.947952
3,3,torture detainees degrading detention interrog...,38.086841
14,14,presentations transcript presentation similar ...,35.041825
22,22,commenting connecting fill wordpress log icon ...,33.995046
16,16,yoo memo memos democrats torture codevilla tor...,32.759921
19,19,sontag hooded fig mitchell ghraib baudrillard ...,31.943709
35,35,penguins chin smile metathesis bikini inches h...,31.326977
7,7,apollo lunar moon buzz aldrin armstrong module...,31.000000


In [19]:
data['words'][9]

'apollo lunar moon buzz aldrin armstrong module landing astronaut nasa neil astronauts surface eagle orbit collins mission pilot crew space'